In [1]:
import pandas as pd 
import numpy as np

from sklearn.model_selection  import train_test_split
from sklearn.preprocessing    import StandardScaler
from sklearn.preprocessing    import MultiLabelBinarizer
from sklearn.metrics          import classification_report, confusion_matrix

from tensorflow.keras.models  import Sequential
from tensorflow.keras.layers  import Dense, Dropout


In [6]:
!pwd
df = pd.read_csv("../../../Z_datasets/auto.csv")
df.head()

/home/sergio/Yandex.Disk/clases/neuronales/BA_pmc


,compactness,circularity,dist_circularity,radius_ratio,pr_axis,max_length_ratio,scatter_ratio,elongatedness,pr_axis_rect,max_leng_regularity,scaled_var_major,scaled_var_minor,scaled_rad_gyration,skewness_major,skewness_minor,kurtosis_minor,kurtosis_major,hollows_ratio,class
0,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
1,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
2,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
3,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
4,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus


Como los datos tienen valores fuera del $[-3,3]$ donde la sigmoide y la tangente hiperbólica funcionan, entonces vamos a escalarlos.

In [10]:
datos =  df.iloc[:,:-1]
datos_s = StandardScaler().fit_transform(datos)
datos_s

array([[ 0.16236242,  0.51220379,  0.05795792, ...,  0.37859592,
        -0.3144815 ,  0.18254814],
       [-0.32429188, -0.62407988,  0.12141055, ...,  0.1547051 ,
         0.01000798,  0.45158325],
       [ 1.25733459,  0.83685627,  1.51736843, ..., -0.40502194,
        -0.15223676,  0.04803059],
       ...,
       [ 1.50066174,  1.48616123,  1.20010528, ..., -0.96474898,
        -0.3144815 ,  0.72061836],
       [-0.93260975, -1.43571108, -0.25930524, ...,  1.38610459,
         0.17225272, -0.08648697],
       [-1.05427333, -1.43571108, -1.02073681, ...,  0.60248673,
        -0.47672624, -0.75907474]])

In [11]:
dummies = pd.get_dummies(df)
clasif  = dummies.iloc[:,-4:]
clasif.shape

(843, 4)

Con esto nuestra variable de clasificación ahora va a estar codificada como un vector indicador donde la i-ésima variable es 1
$$[0,\ldots,1,\ldots,0]$$

Ahora lo que vamos a hacer es dividir nuestro conjunto de aprendizaje en conjunto de prueba y aprendizaje

In [13]:
Xtr, Xts, ytr, yts = train_test_split(datos_s, clasif)
print(Xtr.shape," ", Xts.shape)
print(ytr.shape," ", yts.shape)

(632, 18)   (211, 18)
(632, 4)   (211, 4)


## Keras

Keras es una biblioeteca para implementar redes neuronales que usa como backend al menos dos motores, el más común es tensorflow (Google) de hecho ya se distribuye junto con TF en su versión 2.0.

Consta de dos modalidades para crear capas de redes: Sequential, la más común, que funciona mediante el agregado de capas específicas que llevan a cabo alguna función como una capa de neuronas densas o bien una capa de Dropout, etcétera. La otra modalidad es la de Functional que permite más versatilidad para crear redes.

Un ejemplo muy directo es el siguiente. Para crear una red hay que crear el modelo, agregar las capas necesarias, definir la pérdida y el optimizador. Y al final evaluar el modelo.


    ni = 18 # número de neuronas en la capa de entrada
    nh = 4  # número de neuronas en la capa escondida
    no = 4  # número de neuronas en la capa de salida (cantidad de clases)
    model = Sequential()
    model.add(Dense(nh, input_dim=ni, activation='tanh') )
    model.add(Dense(no, activation='softmax'))   #ultima capa que usa softmax

    #Ahora compilamos con la perdida 'categorical_crossentropy'
    #con optimizador 'adam' -en vez de sgd- y la métrica es 'accuracy'

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Vamos a inspeccionar nuestro modelo

    model.summary()
    
    Model: "sequential_3"
    _________________________________________________________________
    Layer (type)                 Output Shape              Param #   
    =================================================================
    dense_6 (Dense)              (None, 8)                 152       
    _________________________________________________________________
    dense_7 (Dense)              (None, 4)                 36        
    =================================================================
    Total params: 188
    Trainable params: 188
    Non-trainable params: 0
    _________________________________________________________________

Ya que hicimos esto ahora podemos entrenar nuestra red

    model.fit(Xtr, ytr, epochs=300, verbose=1)

Vamos a evaluar 

    model.evaluate(Xts, yts)

    7/7 [==============================] - 0s 1ms/step - loss: 0.3824 - accuracy: 0.8199


Ninguna maravilla pero nuestro modelo ya quedó entrenado.

    predics = model.predict_proba(Xts)

    print(np.round(predics))

Pero si queremos averiguar qué configuración tiene la mejor calificación variando la cantidad de neuronas intermedias podemos meter todo en un ciclo y podemos crear una función que genere la red de forma más sencilla

In [25]:
def crea_modelo(ni, nh, no):
    model = Sequential()
    model.add(Dense(nh, input_dim=ni, activation='relu'))
    model.add(Dense(no, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

nn1 = crea_modelo(ni, 4, no)
nn2 = crea_modelo(ni, 6, no)
nn3 = crea_modelo(ni, 8, no)


Ahora podemos meter todo en un `for` para averiguar cuál es el mejor

In [27]:
n_epocas = 300 
for lbl, model in [('4h', nn1), ('6h', nn2), ('8h',nn3)]:
    print("Modelo con {} escondidas".format(lbl))
    model.fit(Xtr,ytr, epochs=n_epocas, verbose=0)
    model.evaluate(Xts,yts)
    preds = model.predict_proba(Xts)
    reporte = classification_report(yts, np.round(preds))
    print(reporte)

Modelo con 4h escondidas
7/7 [==============================] - 0s 1ms/step - loss: 0.3912 - accuracy: 0.7962
              precision    recall  f1-score   support

           0       0.98      0.96      0.97        50
           1       0.75      0.49      0.59        55
           2       0.62      0.74      0.67        53
           3       0.94      0.96      0.95        53

   micro avg       0.82      0.78      0.80       211
   macro avg       0.82      0.79      0.80       211
weighted avg       0.82      0.78      0.79       211
 samples avg       0.78      0.78      0.78       211

Modelo con 6h escondidas


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


7/7 [==============================] - 0s 913us/step - loss: 0.3423 - accuracy: 0.8199
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        50
           1       0.69      0.62      0.65        55
           2       0.65      0.60      0.63        53
           3       0.95      1.00      0.97        53

   micro avg       0.82      0.80      0.81       211
   macro avg       0.82      0.81      0.81       211
weighted avg       0.81      0.80      0.81       211
 samples avg       0.80      0.80      0.80       211

Modelo con 8h escondidas


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


7/7 [==============================] - 0s 6ms/step - loss: 0.3542 - accuracy: 0.8057
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        50
           1       0.68      0.55      0.61        55
           2       0.63      0.70      0.66        53
           3       0.95      1.00      0.97        53

   micro avg       0.81      0.80      0.80       211
   macro avg       0.81      0.81      0.80       211
weighted avg       0.81      0.80      0.80       211
 samples avg       0.80      0.80      0.80       211



/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


En este caso la mejor red fue la última con 8 neuronas escondidas y función de activación `relu`